In [3]:
# -*- coding: utf-8 -*-
"""
Spyder Editor

This is a temporary script file.
"""
import pandas as pd
import numpy as np
import operator
#import matplotlib.pyplot as plt
#from matplotlib.font_manager import FontProperties
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import pylab as pl
import copy
# from sklearn.model_selection import train_test_split
dataset_path = "/Users/Optimus-Prime/Desktop/cl_train_df.csv"
df = pd.read_csv(dataset_path) # read in data as the DataFrame format

ImportError: No module named sklearn.linear_model

In [ ]:
#数据处理部分---------------
# 数据清洗
#数据分割
#IMbalanced 处理








#特征选取--------------------------------------------------------------------
ranked_feas = list(df.keys()) # get all feature names as a list object
del ranked_feas[0] # delete 'category' column
M_KS={}
M_IV={}
for ikey in ranked_feas:
        files=df[['loan_status', ikey]]
        length=len(files)-1
        file=np.array(files)
        D={}
        sum=0.0
        psum=0.0
        nsum=0.0
        for i in range(0,length):
            [loan_status,score]=file[i]
            if loan_status=="loan_status": continue           
            try:
                D[str(score)][2] +=1
            except:
                D[str(score)]=[0,0,1]
            if loan_status==0:
                D[str(score)][0] +=1
                psum +=1
            else:
                D[str(score)][1] +=1
                nsum +=1
            sum +=1
        num_bins=0            
        dkeys=sorted(list(D.keys()),key=lambda x: x)
        bin_sizes=[]
        ksi=[]
        iv=0.0
        IIV=0.0
        ipsum=0.0
        insum=0.0
        isum=0.0
        iipsum=0.0
        iinsum=0.0
        for iscore in dkeys:
            ipsum += D[iscore][0]
            iipsum += D[iscore][0]
            insum += D[iscore][1]
            iinsum += D[iscore][1]
            isum  += D[iscore][2]
            t=isum/sum
            if t>0.1 :
                bin_sizes += [t]
                num_bins += 1
                ksi += [(np.abs(ipsum/psum-insum/nsum))*100]
                iv += (iipsum/psum-iinsum/nsum)*(np.log((iipsum/psum+0.01)/(iinsum/nsum+0.01)))
                isum=0.0
                t=0.0
                iipsum=0.0
                iinsum=0.0
            else:
                continue
        IIV=iv+(iipsum/psum-iinsum/nsum)*(np.log((iipsum/psum+0.01)/(iinsum/nsum+0.01)))
        ksi += [(np.abs(ipsum/psum-insum/nsum))*100]
        IKS=max(ksi)
        M_KS[ikey]=IKS
        M_IV[ikey]=IIV
        
        

In [ ]:
#计算排序并筛选变量
        
sorted_MKS=sorted(M_KS.items(),key=operator.itemgetter(1),reverse=True)
sorted_MKS=[x for x in sorted_MKS if x[1]>0]
sorted_MIV=sorted(M_IV.items(),key=operator.itemgetter(1),reverse=True)
sorted_MIV=[x for x in sorted_MIV if x[1]>0]
sorted_feas_MKS=[x[0] for x in sorted_MKS]
sorted_feas_MIV=[x[0] for x in sorted_MIV]
sorted_feas=[x for x in sorted_feas_MKS if x in sorted_feas_MIV]
print (sorted_MKS)
print (sorted_MIV)

In [ ]:
#模型搭建----------------------------------------------------------------
#以下进行数据逻辑回归

#selected_df=pd.merge(df['loan_status'],(df[sorted_feas]),on='loan_status')

dum_purpose=pd.get_dummies(selected_df['purpose'],prefix='purpose')
cols_to_keep=ranked_feas.remove('purpose')
#需要根据输入数据进行改变，确认或重写数据清理代码
#dumed_data=df[cols_to_keep].join(dum_purpose.ix[:,'purpose_2':])

#dumed_data['intercept']=1.0

#逻辑回归执行
#train_cols=dumed_data.columns[1:]
#train_cols=selected_df.columns[1:]
#logit=sm.Logit(dumed_data['loan_status'],dumed_data[train_cols])
logit=sm.Logit(df['loan_status'],df[sorted_feas])
logit_result=logit.fit()

In [ ]:
#模型测试部分---------------------------------------------------------------
#数据的分割train和test，放在第一部分，此处临时使用
#variable_train,variable_test,loan_states_train,loan_states_test=train_test_split(variable,loan_states,test_size=0.25,random_state=0)
#loan_states,variable = dumed_data.ix[:,0],dumed_data.ix[:,1:]


combos=copy.deepcopy(df)
predict_cols = combos.columns[1:]  
combos['intercept'] = 1.0 
combos['predict'] = logit_result.predict(combos[predict_cols])
total = 0.0  
hit = 0.0  
for value in combos.values:  
    predict = value[-1]  
  # 实际录取结果  
    admit = int(value[0])  
   
  # 假定预测概率大于0.5则表示预测被录取  
    if predict > 0.5:  
        total += 1  
    # 表示预测命中  
    if admit == 1:  
        hit += 1  
  # 预测分数 predict, 是数据中的最后一列  
  
print 'Total: %d, Hit: %d, Precision: %.2f' % (total, hit, 100.0*hit/total) 

#RF模型